# 1. Import Libraries and Set Directory
---

Import Libraries

In [22]:
is_drive = 0

In [23]:
if is_drive==1 :
    from google.colab import drive
    drive.mount('/content/drive')

In [24]:
# directory management
import os
from glob import glob
# regular imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# imports for NN
import tensorflow as tf
from tensorflow import keras
import time

# 2. Load Dataset
---

In [25]:
# Spectogram Path
if is_drive==1 :   
    TRAIN_DIR = "/content/drive/MyDrive/spectogram-dataset/Train/"
    TEST_DIR = "/content/drive/MyDrive/spectogram-dataset/Test/"
    SPECTROGRAMS_SAVE_DIR = "/content/drive/MyDrive/autoencoder-generations/Test/"
else :
    TRAIN_DIR = "../Project/spectogram-dataset/Train/"
    TEST_DIR = "../Project/spectogram-dataset/Test/"
    SPECTROGRAMS_SAVE_DIR = "../Project/autoencoder-generations/Test/"

In [26]:
def load_fsdd(the_path):
    x_train = []
    for root, _, file_names in os.walk(the_path):
        for file_name in file_names:
            file_path = os.path.join(root, file_name)
            spectrogram = np.load(file_path) # (n_bins, n_frames)
            x_train.append(spectrogram)
    x_train = np.array(x_train)        # -> (n_samples, n_bins, n_frames)
    x_train = x_train[..., np.newaxis] # -> (1200, 800, 32, 1)
    
    return x_train

In [27]:
x_train = load_fsdd(TRAIN_DIR)
x_test  = load_fsdd(TEST_DIR)
AE_test = load_fsdd(SPECTROGRAMS_SAVE_DIR)
AE_test = AE_test[:, :, :, :, 0]

In [28]:
print(x_train.shape)
print(x_test.shape)
print(AE_test.shape)

(1200, 800, 32, 1)
(300, 800, 32, 1)
(300, 800, 32, 1)


In [29]:
num_classes = 10
iterations = 0
i = 0
y_train = []
while iterations!=len(x_train):
    iterations +=1
    y_train.append(i)
    i += 1
    if i == num_classes :
        i = 0
y_train = np.array(y_train)
#---------------------------------------------------------------#
iterations = 0
i = 0
y_test =[]
while iterations!=len(x_test):
    iterations +=1
    y_test.append(i)
    i += 1
    if i == num_classes :
        i = 0
y_test = np.array(y_test)

y_train=keras.utils.to_categorical(y_train,num_classes)
y_test =keras.utils.to_categorical(y_test,num_classes)

In [30]:
print(y_train[0])
print(y_train[5])
print(y_test[9])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


# 3. Create Classifer